# 모델 매개변수 최적화하기

모델과 데이터가 준비되면 데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트할 차례입니다.
모델의 학습과정은 반복적인 과정(epoch)을 통해 추측과 정답 사이의 오류(loss)를 계산하고 오류의 도함수(derivative)를 수집한 뒤,
경사하강법을 사용하여 파라미터들을 최적화합니다. 

### 기본 코드

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

### 하이퍼파라미터(Hyperparameter)

하이퍼파라미터는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수입니다. 이는 학습으로 조절되지는 않지만
하이퍼파라미터의 값은 모델 학습과 수렴에 영향을 미칩니다. 
보통 학습 시에 사용되는 하이퍼파라미터는 아래와 같습니다. 
* epoch - 데이터셋을 반복하는 횟수
* batch size - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수 
* learning rate - 각 배치/에폭 에서 모델의 매개변수를 조절하는 비율. 값을 작으면 학습 속도가 느려지며, 값이 크면 수렴하기 힘들어 질 수 있습니다.    

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### 최적화 단계 

최적화 단계의 각 반복을 epoch이라 부르며 
하나의 epoch은 두 부분으로 구성됩니다.
* 학습 단계 (train loop) - 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴합니다. 
* 검증/테스트 단계 - 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복합니다. 


### 손실함수(loss function)

손실함수는 모델이 획득한 결과와 실제 값 사이의 틀린 정도를 측정하며, 학습 중에는 이 값을 최소화하는 방향으로 학습을 진행합니다. 
일반적인 손실함수는 아래와 같습니다. 
* 회귀문제(regression) - nn.MSELoss
* 분류문제(classification) - nn.NLLLoss, nn.CrossEntropyLoss


In [3]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

### Optimizer

최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정입니다. 최적화 알고리즘은 이 과정이 수행되는 방식(여기에서는 확률적 경사하강법(SGD; Stochastic Gradient Descent))을 정의합니다.
모든 최적화 절차는 optimizer 객체에 캡슐화 됩니다. 


In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습 단계(loop)에서 최적화는 세단계로 이뤄집니다:
* optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정합니다. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정합니다.
* loss.backward()를 호출하여 예측 손실(prediction loss)을 역전파합니다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장합니다.
* 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다.

### 전체 구현

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
#손실함수와 옵티마이저를 초기화하고 train loop와 test loop에 전달합니다. 
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296058  [    0/60000]
loss: 2.291445  [ 6400/60000]
loss: 2.279195  [12800/60000]
loss: 2.281773  [19200/60000]
loss: 2.265295  [25600/60000]
loss: 2.241428  [32000/60000]
loss: 2.252755  [38400/60000]
loss: 2.228477  [44800/60000]
loss: 2.214210  [51200/60000]
loss: 2.209185  [57600/60000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 2.206954 

Epoch 2
-------------------------------
loss: 2.195545  [    0/60000]
loss: 2.192936  [ 6400/60000]
loss: 2.163235  [12800/60000]
loss: 2.196950  [19200/60000]
loss: 2.142715  [25600/60000]
loss: 2.097704  [32000/60000]
loss: 2.136948  [38400/60000]
loss: 2.081262  [44800/60000]
loss: 2.070784  [51200/60000]
loss: 2.053642  [57600/60000]
Test Error: 
 Accuracy: 48.9%, Avg loss: 2.050915 

Epoch 3
-------------------------------
loss: 2.029916  [    0/60000]
loss: 2.020678  [ 6400/60000]
loss: 1.960731  [12800/60000]
loss: 2.041980  [19200/60000]
loss: 1.926240  [25600/60000]
loss: 1.857210  [32000/600